In [1]:
import pandas as pd

In [2]:
df = pd.read_json('output.json')

In [4]:
df['texto'] = df['texto'].apply(lambda x: ''.join(x))

In [9]:
print(df.iloc[20]['texto'])

Processo Nº 001231 / 2023 - TC (001231/2023-TC)
SESSÃO ORDINÁRIA 00004ª, DE 15 DE FEVEREIRO DE 2024 - 1ª CÂMARA.
Interessado(s):   PREFEITURA MUNICIPAL DE NÍSIA FLORESTA/RN                                                                                                                                                                                                                      
  
Assunto:  DENÚNCIA
Relator(a):  MARCO ANTÔNIO DE MORAES RÊGO MONTENEGRO
EMENTA: DENÚNCIA. AUSÊNCIA DE LASTRO PROBATÓRIO MÍNIMO PARA 
O DESENVOLVIMENTO REGULAR DO PROCESSO.  ADMISSIBILIDADE 
PREJUDICADA. INTELIGÊNCIA DO ARTIGO 80, § 1º, DA LOTCE . 
ARQUIV AMENTO DO PRESENTE FEITO.
ACÓRDÃO No. 40/2024 - TC
               Vistos, relatados e discutidos estes autos, em consonância ao posicionamento do 
Corpo técnico e do Ministério Público de Contas, ACORDAM os Conselheiros, nos termos 
do voto proposto pelo Conselheiro Relator, julgar a inadmissibilidade da presente denúncia e 
o seu conseqüente arquivamen